# Respuestas a preguntas de cuaderno.

El presente documento tiene la finalidad de responder las preguntas asociadas al cuaderno **Notas de revisión GO3**, esto con la finalidad de evaluar si es válido o no utilizar este paquete dentro de los proyectos del laboratorio. Además de establecer consideraciones sobre como debe ser usado. En caso de requerirlo, se proporciona el código fuente asociado a algún proceso intermedio que pueda ser de utilidad para proximos proyectos.

## ¿Qué tipo de representación se utilizan para la identificación de genes? ¿Cuáles son compatibles y como afectan al resultado?

Una cosa relevante a considerar es que representación se utiliza para determinar las entidades a las cuales se les aplica el cálculo de cualquier métrica de similitud:
   1. **Términos**: _Gene Ontology Terms_ presentes en el archivo "go-basic.obo", estos se presentan mediante la nomenclatura GO:[ID]. Además que el archivo fuente proporciona las conexiones _is_a_ & _part_of_ para la construcción del GoDAG, es decir, el arbol aciclíco no dirigino.
   2. **Genes**: Estos se definen formalmente como secciones del ADN las cuales representan la codificación de un producto funcionas, es decir, proteinas encargadas de proocesos dentro de un organismo. Dado a que existen varías organizaciones, determinar que existe un único identificador disponible es complejo de determinar (aunque lo más cercano es lo que se llama EntrezID).

Como forma de comprender cual es el identificador de los genes se decidió estudiar uno de los archivos más relevante existente dentro del proceso de estudio de métricas de similitud _goa-human.gaf_, el cual es un archivo que **vincula el genes con sus respectivos términos de _Gene Ontology_** y que, guiados por archivos _.obo_, permiten realizar una relación entre los genes de forma que se pueda estudiar su similitud a nivel de estos componentes. Dado a que su estructura es parecido a un archivo de columnas separadas por tabulaciones se pueden reconocer los siguientes elementos:
   1. Base de datos:       Base de datos de referencia para determinar relación entre genes.
   2. DB Object ID:        Identificador único de la proteína o gen.
   3. DB Object Symbol:    Simbolo simplificado del objeto.
   4. Qualifier:           Determina contribución del término en las relaciones.
   5. GO ID:               Identificador del termino GO asociado.
   6. DB:Reference:        Referencia que respalda Gen con anotación
   7. Evidence Code:       Describe como se obtuvo la anotación
   8. With / From:         ID usados para inferir anotación,
   9. Aspect:              Tipo de anotación (ontología)
   10. DB Object Name:     Nombre de gen o proteína-
   11. DB Object Synonym:  Sinonimos del nombre.
   12. DB Object Type:     Tipo de objeto.
   13. Taxon:              Taxonomia.
   14. Date:               Fecha de anotación.
   15. Assigned By:        Entidad de registro.

En terminos prácticos esto solo implica que buscamos determinar la simbología del objeto dentro de solamente el identificador asociado, por ende, he decidido crear una función de apoyo que permita transformar cualquier identificador de genes al que corresponde según el archivo que se ha presentado. Para los ejemplos he decidido usar un conjunto de 500 genes del ser humano y 133 genes de la planta Adhopsis Thaliana como forma de presenta la persistencia de la función según taxonomía.

In [1]:
%%time
%pip install mygene

Note: you may need to restart the kernel to use updated packages.
CPU times: total: 0 ns
Wall time: 2.2 s



[notice] A new release of pip is available: 24.3.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%%time
import pandas as pd
from mygene import MyGeneInfo
import re

def mapeo_genes(
    genes_ids,
    gaf_path,
    scopes_default=("uniprot", "entrezgene", "ensembl.gene", "symbol", "name"),
    fields="symbol"
):
    """
    Normaliza una lista de identificadores génicos hacia símbolos canónicos,
    usando un archivo GAF como referencia primaria.
    """

    # ─────────────────────────────
    # 1. Leer GAF
    # ─────────────────────────────
    gaf = pd.read_csv(
        gaf_path,
        sep="\t",
        comment="!",
        header=None,
        dtype=str
    )

    gaf.columns = [
        "DB", "DB_ID", "Symbol", "Qualifier", "GO_ID",
        "Reference", "Evidence", "With_From", "Aspect",
        "Name", "Synonyms", "Type", "Taxon", "Date", "Assigned_By", 
        "Annotation_Extension", "Gene_Product_Form_ID"
    ]

    # ─────────────────────────────
    # 2. Conjuntos de referencia
    # ─────────────────────────────
    valid_symbols = set(gaf["Symbol"].dropna().unique())
    db_to_scope = {
        "UniProtKB": "uniprot",
        "NCBI_GeneID": "entrezgene",
        "Ensembl": "ensembl.gene"
    }

    # detectar especie desde taxon
    taxon = (
        gaf["Taxon"]
        .dropna()
        .iloc[0]
        .replace("taxon:", "")
    )

    mg = MyGeneInfo()

    # ─────────────────────────────
    # 3. Separar IDs válidos / inválidos
    # ─────────────────────────────
    genes_validos = []
    genes_a_mapear = []

    for g in genes_ids:
        if g in valid_symbols:
            genes_validos.append(g)
        else:
            genes_a_mapear.append(g)

    # ─────────────────────────────
    # 4. Mapear solo los no válidos
    # ─────────────────────────────
    mapping = {}

    if genes_a_mapear:
        res = mg.querymany(
            genes_a_mapear,
            scopes=scopes_default,
            fields=fields,
            species=taxon
        )

        for r in res:
            if not r.get("notfound", False):
                mapping[r["query"]] = r.get("symbol")

    # ─────────────────────────────
    # 5. Construir salida final
    # ─────────────────────────────
    final_mapping = {}

    for g in genes_ids:
        if g in genes_validos:
            final_mapping[g] = g
        else:
            final_mapping[g] = mapping.get(g)

    lista_final = list(
        {v for v in final_mapping.values() if v is not None}
    )

    return {
        "mapping": final_mapping,
        "final_gene_list": lista_final,
        "species_taxon": taxon
    }

# Ejemplo humano y Thaliana.
ids_human = ["VSTM2L","N4BP1","GIT2","FAM160A2","OMA1","TBCK","CCDC47","STAG2","CCPG1",
             "ETAA1","HOOK1","PDK4","LMBRD1","MAGEF1","C1QTNF1","PPP2R2D","PIK3AP1",
             "GLIPR1","TMBIM4","MS4A1","FBLIM1","ARID1A","ACVRL1","MAN1C1","TMEM129",
             "KRT23","RDH11","NUP54","ZBTB7A","LIN7C","EGFL7","ABHD4","AMDHD2","RNF146",
             "DOCK7","STARD4","RCOR3","CAMTA1","COX18","SP1","AKT1S1","LONP2","THNSL2",
             "XPO4","TSPAN15","FBXO30","ANGPTL4","ARHGAP24","SSH1","ECHDC2","TMX3","GNA13",
             "SVIP","ZC3H12A","UQCR10","HEATR5A","CCM2","DYNLRB1","TM7SF3","ATAD1","FAM53C",
             "TMEM167A","CMTM6","RHOBTB3","IRS2","SLC44A2","NPDC1","ASAP1","NUS1","SCYL1",
             "DHFR","KPNA4","UFSP2","AIG1","SPCS3","APPL2","TMEM50A","TRAPPC5","ING4",
             "GINS2","ITM2B","NDUFA4","OST4","RPS27L","NAA20","UBL5","DNAJC10","SF3B5",
             "ALKBH7","MYOZ1","EIF2A","ERRFI1","PACS1","PCYOX1","HVCN1","STYXL1","GATAD2A",
             "BRWD1","GPM6B","KLF11","EIF4ENIF1","USP39","NSFL1C","ARRDC1","PFDN2","STARD3NL",
             "RANGRF","ECHDC1","PPTC7","TMED4","WIPF2","ARHGAP17","CAPRIN2","ENAH","RLIM",
             "RAB25","DDR2","BAG3","TPCN2","WBP11","ESAM","ANAPC16","PBRM1","SLC44A1","ERGIC1",
             "GOLPH3","HAUS1","NDUFB10","MYLIP","HSD17B12","GNAQ","ECSIT","DENR","PPHLN1",
             "UBQLN1","PYCR2","NAA50","DDOST","IER3IP1","MRPL55","NEXN","PAPPA","PRKCE","ABHD12",
             "DTX3","AK3","UBE2F","B3GALT6","PPA2","PIH1D1","PIM3","RAB10","NSMCE1","UFM1",
             "PPME1","TIMP2","VPS29","HMG20A","CRK","ACACA","RIC8A","DDAH1","PIGT","ADAMTS2",
             "ZNF358","RASGRP4","SOCS3","MRPS22","GET4","B3GAT2","SMAP1","MRPS15","MACROD1",
             "SEC61A2","VWA1","HM13","FOXK1","MAFK","AP2A1","FUZ","BAZ1A","ARID2","PCGF3",
             "RRN3P2","KIF5A","ORMDL2","SLTM","NLRC3","NR2F6","CBLC","TAF1D","SLC2A4RG",
             "THOC7","RREB1","EPB41","UBR5","EXOSC3","IRF2BP2","KLHL22","ZNF224","CDC42SE1",
             "USP13","LY75","WWC3","UBR4","FECH","RPS6KA6","TBX5","COL17A1","HK2","SPRED1",
             "LYST","PRRC1","AMMECR1","ZNF117","GLIS3","DMRT3","ENOPH1","BEND3","MRPS26",
             "NDUFC2","ZNF497","PPP2R2C","RNF34","RP9","GON4L","MSTO1","SEMA6D","USP42",
             "PPP1R3F","AGAP3","ALG5","MYBBP1A","HES6","MTHFD1L","PGLS","PTPMT1","FOXO3",
             "NR2C2","MALAT1","ZNF672","MMP24","VAMP8","ZRANB1","ISYNA1","RNF169","HCN3",
             "ZBTB46","DAGLB","LGALS8","ARID1B","SPAG9","CERCAM","EFNA5","TAF6L","TRIM8",
             "TMEM165","UBXN6","MTFP1","SGTB","PET117","SSU72","TLE3","CAMK2N2","ECE2","DOCK6",
             "RWDD2B","SLC25A37","CRLS1","IREB2","AKAP12","PIK3R5","GPR12","PHF8","STX11","DLC1",
             "USP53","HELZ","DNAH14","PTBP2","KIF13A","TRPC5","KRT82","SNX25","PSMD13","HLCS","AGGF1",
             "BRWD3","CD28","PTCHD1","UQCRB","DYNC1H1","CLN6","MED26","PTPRS","SNX14","ZDHHC22","MTPAP",
             "ANAPC7","MAT2B","WDR4","B3GNT9","ZBTB26","NXF3","TUBE1","KCTD5","DDX54","TRIM69","PPCS",
             "KANK4","C1orf56","GADD45GIP1","TIMM22","SLC33A1","FOXH1","SAMD1","MLXIPL","MRPL35","EGLN2",
             "PARP8","PHF20","NCS1","E2F6","NBR1","ALG13","ERI1","MYO9A","SLC6A11","GNAT1","KLHL18","COL13A1",
             "HIGD1C","FAM9B","CAND1","CAST","SMAD9","MDM4","RBM12","ZNF626","RGMB","GPATCH2","NAP1L4",
             "LSM2","DLG1","INPP5F","NRXN1","RELL1","ZFPM2","SNRPN","AKAP5","ZNF148","CDK19","VGLL3",
             "USP8","TACC2","MRPS10","NR2C2AP","FA2H","DAZAP1","CCDC136","SAP18","DCLRE1C","RPS23","IP6K2",
             "SCML1","SHARPIN","TM9SF1","TOX4","GPRC5B","MRPL10","NMRAL1","PLIN3","TP53I13","TES","PSAT1",
             "BCL2L10","HECTD2","SERP1","CRIPT","THAP3","RPS3A","SMAD3","MPRIP","NUP107","SPRYD3","SNX12",
             "PANX1","RPRM","ABCA7","MDFIC","SAE1","CLCN2","TTC17","TTC19","SPESP1","MTRF1L","GALNT1",
             "FBXO8","NUDCD2","VIL1","UGCG","TMEM119","IL1RAP","RAPGEF1","RNF126","PPP2R3B","ABCG8",
             "TXNDC16","ZNF704","RAD50","CORO7","OSR2","TEKT5","RFX4","LGALS16","MED13L","THOC3","PSMA8",
             "TBX4","ZNF655","TMEM132B","KDM5D","KCNJ16","INSC","RABGAP1","HSBP1L1","MKRN1","CALM1","MCPH1",
             "SLC39A13","NEFL","KDM2B","MAGED2","PAPOLA","DDX20","THAP7","LGMN","DAB2IP","ACTG1","SNX9",
             "DNAJC4","TTC39C","FPR3","NADSYN1","TYW1","UBE2D4","ZCCHC3","ENSA","VTA1","PSMD1","BRD3",
             "TRDN","RSL1D1","ERMN","TERF1","B3GALNT2","BMPR2","TMSB10","FLYWCH2","YME1L1","SURF4","HNRNPL",
             "PLSCR4","TBRG4","PLEKHB2","STXBP5L","PCDHB15","KCNT2","WDR90","B4GALNT1","DBT","LNPEP","ZIC4",
             "MMAA","CELF3","NUP155","ARID5B","COQ2","AGPAT3","KCNH8","RPL4","KYNU","GNAL","DAP3","NRBF2","CARD17",
             "COL4A2","BEND5","BMP2K","NEK10","STAB2","NDUFV3","RAB28","MATR3","XPNPEP1","MAPK9","VNN1",
             "CETN1","INTS6","SETD4"]

ids_thaliana = ["AT4G16610","AT4G29010","AT2G34420","AT2G43080","AT3G12120","AT3G04720","AB000623","AT1G55020",
                "T04323","AT1G76100","AT3G15730","AT3G12500","AT5G38430","AT1G33960","AT1G32060","AT5G65110",
                "AT1G28670","AT4G39260","AT2G24180","AT2G26740","AT5G35630","AT3G15350","AT2G35790","AT3G06860",
                "Y11607","AT2G34660","AT2G33150","AT2G23430","AT2G39780","AT4G25100","AB003040","AT1G78380","AT1G37130",
                "Z56278","AT1G07890","X66017","AT4G39330","AT5G13930","AT3G20230","AT4G39980","AT1G31580","AT4G37980",
                "AT2G34770","AJ000470","L15389","AF062901","AT4G00860","AT3G57260","AT4G20360","AT3G16480","AT3G16520",
                "AT2G42010","AT2G02990","AT1G76490","X99793","AT4G38530","AT1G67980","AT5G39190","AT3G02520","AT3G59700",
                "AT4G09320","AT3G44300","AT5G24090","AT3G25230","AT1G08830","AT1G51680","AT1G05010","AT4G16760","AT5G42650",
                "AT5G05730","AT1G25220","AT3G29200","AT4G31500","AT3G13920","AT3G11170","Z26426","AT2G29450","AF087932",
                "AT5G02500","AT1G51760","AT3G45140","AT1G76680","AT2G37040","AT3G53260","AT1G56190","Y14590","AT3G30775",
                "AT5G42180","AT4G13940","AT3G54640","AT5G54810","AT3G50970","AT3G18780","AT1G16060","AT5G54160","X16432",
                "AT1G64280","AT5G66570","AT4G01850","AT1G01470","AT2G20340","AT3G47290","AT2G14610","AT1G75040","AT5G20620",
                "AT2G26330","AT2G25080","AT4G14980","AT1G67750","AT1G67550","AT5G26000","AT5G25980","AT2G05510","AT1G04820",
                "AT1G01480","AT3G28930","AT4G20890","AT2G45820","AT3G15210","AT5G47770","AT2G28740","AT2G14850","AT4G08500",
                "AT3G45640","AT5G52350","AT3G20600","AT5G06940","AT1G53840","AT2G41110","AT5G37770","AT2G41100","AT5G57560",
                "AT1G72260"]

res = mapeo_genes(ids_human, gaf_path=r"C:\Users\benja\Desktop\workspace\GO3_Study\Files\goa_human.gaf")

print(res["mapping"])
print(res["final_gene_list"])
print(len(res["final_gene_list"]))
print(res["species_taxon"])

res = mapeo_genes(ids_thaliana, gaf_path=r"C:\Users\benja\Desktop\workspace\GO3_Study\Files\tair.gaf", scopes_default=["tair", "symbol", "ensembl.gene", "uniprot"])
print(res["mapping"])
print(res["final_gene_list"])
print(len(res["final_gene_list"]))
print(res["species_taxon"])


Input sequence provided is already in string format. No operation performed
1 input query terms found dup hits:	[('MALAT1', 8)]
2 input query terms found no hit:	['FAM160A2', 'CARD17']


{'VSTM2L': 'VSTM2L', 'N4BP1': 'N4BP1', 'GIT2': 'GIT2', 'FAM160A2': None, 'OMA1': 'OMA1', 'TBCK': 'TBCK', 'CCDC47': 'CCDC47', 'STAG2': 'STAG2', 'CCPG1': 'CCPG1', 'ETAA1': 'ETAA1', 'HOOK1': 'HOOK1', 'PDK4': 'PDK4', 'LMBRD1': 'LMBRD1', 'MAGEF1': 'MAGEF1', 'C1QTNF1': 'C1QTNF1', 'PPP2R2D': 'PPP2R2D', 'PIK3AP1': 'PIK3AP1', 'GLIPR1': 'GLIPR1', 'TMBIM4': 'TMBIM4', 'MS4A1': 'MS4A1', 'FBLIM1': 'FBLIM1', 'ARID1A': 'ARID1A', 'ACVRL1': 'ACVRL1', 'MAN1C1': 'MAN1C1', 'TMEM129': 'TMEM129', 'KRT23': 'KRT23', 'RDH11': 'RDH11', 'NUP54': 'NUP54', 'ZBTB7A': 'ZBTB7A', 'LIN7C': 'LIN7C', 'EGFL7': 'EGFL7', 'ABHD4': 'ABHD4', 'AMDHD2': 'AMDHD2', 'RNF146': 'RNF146', 'DOCK7': 'DOCK7', 'STARD4': 'STARD4', 'RCOR3': 'RCOR3', 'CAMTA1': 'CAMTA1', 'COX18': 'COX18', 'SP1': 'SP1', 'AKT1S1': 'AKT1S1', 'LONP2': 'LONP2', 'THNSL2': 'THNSL2', 'XPO4': 'XPO4', 'TSPAN15': 'TSPAN15', 'FBXO30': 'FBXO30', 'ANGPTL4': 'ANGPTL4', 'ARHGAP24': 'ARHGAP24', 'SSH1': 'SSH1', 'ECHDC2': 'ECHDC2', 'TMX3': 'TMX3', 'GNA13': 'GNA13', 'SVIP': 'SVIP

Input sequence provided is already in string format. No operation performed
14 input query terms found no hit:	['AB000623', 'T04323', 'Y11607', 'AB003040', 'Z56278', 'X66017', 'AJ000470', 'L15389', 'AF062901', '


{'AT4G16610': 'AT4G16610', 'AT4G29010': 'AIM1', 'AT2G34420': 'LHB1B2', 'AT2G43080': 'AT-P4H-1', 'AT3G12120': 'FAD2', 'AT3G04720': 'PR4', 'AB000623': None, 'AT1G55020': 'LOX1', 'T04323': None, 'AT1G76100': 'PETE1', 'AT3G15730': 'PLDALPHA1', 'AT3G12500': 'HCHIB', 'AT5G38430': 'RBCS1B', 'AT1G33960': 'AIG1', 'AT1G32060': 'PRK', 'AT5G65110': 'ACX2', 'AT1G28670': 'ARAB-1', 'AT4G39260': 'CCR1', 'AT2G24180': 'CYP71B6', 'AT2G26740': 'SEH', 'AT5G35630': 'GS2', 'AT3G15350': 'AT3G15350', 'AT2G35790': 'AT2G35790', 'AT3G06860': 'MFP2', 'Y11607': None, 'AT2G34660': 'ABCC2', 'AT2G33150': 'PKT3', 'AT2G23430': 'ICK1', 'AT2G39780': 'RNS2', 'AT4G25100': 'FSD1', 'AB003040': None, 'AT1G78380': 'GSTU19', 'AT1G37130': 'NIA2', 'Z56278': None, 'AT1G07890': 'APX1', 'X66017': None, 'AT4G39330': 'CAD9', 'AT5G13930': 'TT4', 'AT3G20230': 'AT3G20230', 'AT4G39980': 'DHS1', 'AT1G31580': 'ECS1', 'AT4G37980': 'ELI3-1', 'AT2G34770': 'FAH1', 'AJ000470': None, 'L15389': None, 'AF062901': None, 'AT4G00860': 'ATOZI1', 'AT3G57

Con esto ya comprendemos que se puede generar una función con la capacidad de mapear los simbolos requeridos dentro de pandas como de mapearlo por medio de herramoientas, por ende, es imperativo que para realizar el cálculo gen por gen sea necesario mapear los genes para que el paquete de prueba funcione. Esto de forma parcial responde las preguntas 2, 3 y 5 que se plantearon en la nota anterior, ya que:
 2. Es obligatorio ocupar la combinación de archivos dada para que el paquete funcione, de lo contrario solo se peca de tener errores dentro del flujo de código o matrices en 0.
 3. Si, es compatible con el uso de resultados en otras especies, incluso, en una comprobación más adelante se verá esta capacidad.
 5. Existe forma de administrar los identificadores de genes, en el campo anterior ya se demostró que si. Por lo tanto, solamente queda construir la función como corresponde.  

## Existen otros paquetes en R ¿Qué diferencias de velocidad y resultados proveen? ¿Son similares?

Para esto se provee un estudio de 4 casos fundamentales en donde se comprueba si en el mejor de los casos los resultados son iguales o similares entre ambas funcionalidades. Para la realización de este estudio se utilizó el cuaderno **Estudio de genes con R** con la finalidad de generar los arhcivos para comprobar la diferencia entre resultados a nivel de términos y a niveles de genes.
Caso 1: 100 terminos de gene ontology (al azar).
Caso 2: 10 pares de genes distintos (humano).
Caso 3: 10 pares de genes distintos (thaliana).
Caso 4: 2 pares de genes (no encontrado por función) - prueba de persistencia.

In [3]:
import go3
from itertools import combinations

# Load Gene Ontology (GO) terms and annotations
go_terms = go3.load_go_terms(r"C:\Users\benja\Desktop\workspace\GO3_Study\Files\go-basic.obo")
annotations_h = go3.load_gaf(r"C:\Users\benja\Desktop\workspace\GO3_Study\Files\goa_human.gaf")

# Build IC Counter
counter = go3.build_term_counter(annotations_h)

terms = ["GO:0098596","GO:0098720","GO:1990000","GO:0007497","GO:0003429","GO:2000348","GO:0009201","GO:0045774","GO:0014877","GO:0030241"]
pairs = list(combinations(terms, 2))
print(pairs)
scores = go3.batch_similarity([a for a, _ in pairs], [b for _, b in pairs], "wang", counter)
print(scores)

sim = go3.semantic_similarity("GO:0098596", "GO:0098720", 'wang', counter)
print(sim)

[('GO:0098596', 'GO:0098720'), ('GO:0098596', 'GO:1990000'), ('GO:0098596', 'GO:0007497'), ('GO:0098596', 'GO:0003429'), ('GO:0098596', 'GO:2000348'), ('GO:0098596', 'GO:0009201'), ('GO:0098596', 'GO:0045774'), ('GO:0098596', 'GO:0014877'), ('GO:0098596', 'GO:0030241'), ('GO:0098720', 'GO:1990000'), ('GO:0098720', 'GO:0007497'), ('GO:0098720', 'GO:0003429'), ('GO:0098720', 'GO:2000348'), ('GO:0098720', 'GO:0009201'), ('GO:0098720', 'GO:0045774'), ('GO:0098720', 'GO:0014877'), ('GO:0098720', 'GO:0030241'), ('GO:1990000', 'GO:0007497'), ('GO:1990000', 'GO:0003429'), ('GO:1990000', 'GO:2000348'), ('GO:1990000', 'GO:0009201'), ('GO:1990000', 'GO:0045774'), ('GO:1990000', 'GO:0014877'), ('GO:1990000', 'GO:0030241'), ('GO:0007497', 'GO:0003429'), ('GO:0007497', 'GO:2000348'), ('GO:0007497', 'GO:0009201'), ('GO:0007497', 'GO:0045774'), ('GO:0007497', 'GO:0014877'), ('GO:0007497', 'GO:0030241'), ('GO:0003429', 'GO:2000348'), ('GO:0003429', 'GO:0009201'), ('GO:0003429', 'GO:0045774'), ('GO:0003

In [4]:
import go3
from itertools import combinations

# Load Gene Ontology (GO) terms and annotations
go_terms = go3.load_go_terms(r"C:\Users\benja\Desktop\workspace\GO3_Study\Files\go.obo")
annotations_h = go3.load_gaf(r"C:\Users\benja\Desktop\workspace\GO3_Study\Files\goa_human.gaf")

# Build IC Counter
counter = go3.build_term_counter(annotations_h)

terms = ["VSTM2L","N4BP1","GIT2","OMA1","AP2A1","SSU72","KLHL18","FOXH1","MRPS15","TTC19"]
pairs = list(combinations(terms, 2))
print(pairs)
scores = go3.compare_gene_pairs_batch(pairs, "BP", "wang", "bma", counter)
print(scores)

sim = go3.compare_genes("VSTM2L", "N4BP1", "BP", "wang", "bma", counter)
print(sim)

[('VSTM2L', 'N4BP1'), ('VSTM2L', 'GIT2'), ('VSTM2L', 'OMA1'), ('VSTM2L', 'AP2A1'), ('VSTM2L', 'SSU72'), ('VSTM2L', 'KLHL18'), ('VSTM2L', 'FOXH1'), ('VSTM2L', 'MRPS15'), ('VSTM2L', 'TTC19'), ('N4BP1', 'GIT2'), ('N4BP1', 'OMA1'), ('N4BP1', 'AP2A1'), ('N4BP1', 'SSU72'), ('N4BP1', 'KLHL18'), ('N4BP1', 'FOXH1'), ('N4BP1', 'MRPS15'), ('N4BP1', 'TTC19'), ('GIT2', 'OMA1'), ('GIT2', 'AP2A1'), ('GIT2', 'SSU72'), ('GIT2', 'KLHL18'), ('GIT2', 'FOXH1'), ('GIT2', 'MRPS15'), ('GIT2', 'TTC19'), ('OMA1', 'AP2A1'), ('OMA1', 'SSU72'), ('OMA1', 'KLHL18'), ('OMA1', 'FOXH1'), ('OMA1', 'MRPS15'), ('OMA1', 'TTC19'), ('AP2A1', 'SSU72'), ('AP2A1', 'KLHL18'), ('AP2A1', 'FOXH1'), ('AP2A1', 'MRPS15'), ('AP2A1', 'TTC19'), ('SSU72', 'KLHL18'), ('SSU72', 'FOXH1'), ('SSU72', 'MRPS15'), ('SSU72', 'TTC19'), ('KLHL18', 'FOXH1'), ('KLHL18', 'MRPS15'), ('KLHL18', 'TTC19'), ('FOXH1', 'MRPS15'), ('FOXH1', 'TTC19'), ('MRPS15', 'TTC19')]
[0.11966790842216044, 0.14409522680594153, 0.10148890293451272, 0.07041064107984213, 0.056

In [3]:
import go3
from itertools import combinations

# Load Gene Ontology (GO) terms and annotations
go_terms = go3.load_go_terms(r"C:\Users\benja\Desktop\workspace\GO3_Study\Files\go.obo")
annotations = go3.load_gaf(r"C:\Users\benja\Desktop\workspace\GO3_Study\Files\tair.gaf")
print(dir(annotations[0]))
print(annotations[0].db_object_id)
print(annotations[1].db_object_id)

# Build IC Counter
counter = go3.build_term_counter(annotations)
print(counter.total_by_ns)
print(counter.counts)
print(counter.ic)

genes = ["ELI3-1","CAM2","CCR1","TSA1","ECS1","HOG1","AT-P4H-1","HLECRK","LTI30"]
pairs = list(combinations(genes, 2))
print(pairs)
scores = go3.compare_gene_pairs_batch(pairs, "BP", "wang", "bma", counter)
print(scores)

sim = go3.compare_genes('MDN1','LTI30', "BP", "wang", "bma", counter)
print(sim)

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'db_object_id', 'evidence', 'go_term']
AT1G67120
AT2G40430
{'biological_process': 24300, 'cellular_component': 37120, 'molecular_function': 24742}
{'GO:0009535': 420, 'GO:0030834': 13, 'GO:0051748': 5, 'GO:0006122': 13, 'GO:0051127': 9, 'GO:0005768': 411, 'GO:0043450': 21, 'GO:0005667': 179, 'GO:0097602': 28, 'GO:0015288': 5, 'GO:0052717': 1, 'GO:0010138': 2, 'GO:0042401': 72, 'GO:0072666': 53, 'GO:1901332': 12, 'GO:0006415': 16, 'GO:1902531': 95, 'GO:0002061': 1, 'GO:0042407': 2, 'GO:0030244': 71, 'GO:0046975': 8, 'GO:0071326': 17, 'GO:0010029': 123, 'GO:0046131': 13, 'GO:0000394': 13, 'GO:0030931': 1, 'GO:0009092': 8, 'GO:0016762': 32, 'GO:0000462': 25, 'G